### Human in the Loop
- Approval need by human to the chatbot to go ahead with the execution

In [ ]:
import os
from langchain.chat_models import init_chat_model
llm = init_chat_model("groq:llama3-8b-8192")
llm

In [ ]:
from typing import Annotated
from langchain_tavily import TavilySearch
from langchain_core.tools import tool 
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

from langgraph.types import Command, interrupt

In [ ]:
# State class
class State(TypedDict):
    messages:Annotated[list,add_messages]

graph_builder = StateGraph(State)
memory = MemorySaver()

@tool
def human_assistance(query:str)->str:
    """Request Assistance from Human

    Args:
        query (str): The Query Chatbot have.

    Returns:
        str: Human Response
    """
    human_response = interrupt({"query":query})
    return human_response["data"]


tool = TavilySearch(max_results=1)
tools = [tool, human_assistance]
llm_with_tools = llm.bind_tools(tools)

# chatbot 
def chatbot(state:State):
    """The main chat bot llm with tools bot with human in the loop, Disable the parallel tool calling to avoid repeating any 
    tool invocations when we resume.
    Args:
        state (State): The input state of the bot
    """
    message = llm_with_tools.invoke(state["messages"])
    return {"messages":message}


# building the flow now in the graph 
graph_builder.add_node("chatbot",chatbot);
tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools",tool_node)
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition
)

#graph builder adding edges
graph_builder.add_edge("tools","chatbot")
graph_builder.add_edge(START,"chatbot")
graph_builder.add_edge("chatbot",END)

#graph builder.compile

graph = graph_builder.compile(checkpointer=memory)


def display_graph(g):
    from IPython.display import Image, display
    try:
        display(Image(g.get_graph().draw_mermaid_png()))
    except Exception:
        pass

display_graph(graph)

In [ ]:
user_input = "I need some expert guidance for building an AI agent. Could you request assistance for me?"
config = {"configurable":{"thread_id":"1"}}

events = graph.stream({"messages":user_input}, config, stream_mode="values")

for event in events:
    if "messages" in event:
        event['messages'][-1].pretty_print()

In [ ]:
human_response = (
    "We are happy to help you out checkout langgraph website and docs"
    "It's much more reliable and extensible than simple autonomous agent."
)

human_command = Command(resume={"data":human_response})

events = graph.stream(human_command,config,stream_mode="values")

for event in events:
    if "messages" in event:
        event['messages'][-1].pretty_print()